<a href="https://colab.research.google.com/github/MuhammedAshraf2020/implementation-of-CNN-Models/blob/main/LeNet/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libs

In [2]:
import torch
import keras
import numpy as np
import torch.optim as torch_optims
import torch.nn as nn

In [3]:
#Load models from github (LeNet using pytorch , LeNet using Tensorflow)

In [4]:
!rm -r /content/LeNet
!rm -r /content/implementation-of-CNN-Models

In [5]:
!git clone https://github.com/MuhammedAshraf2020/implementation-of-CNN-Models/

Cloning into 'implementation-of-CNN-Models'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 169 (delta 93), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (169/169), 36.83 KiB | 5.26 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [6]:
cp -r /content/implementation-of-CNN-Models/LeNet /content/

In [7]:
from LeNet.LeNet_pt import LeNet_pt
from LeNet.LeNet_tf import LeNet_tf
from LeNet.data_tf import data_tf
from LeNet.data_pt import data_pt

In [8]:
# BUILD  TENSORFLOW MODEL

In [9]:
tf_model = LeNet_tf()

In [10]:
tf_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 128)         51328     
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 84)                1

In [11]:
# Build Model using pytorch

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pt_model = LeNet_pt().to(device = device)

In [13]:
print(pt_model)

LeNet_pt(
  (ConvModel): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  )
  (DenseModel): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [14]:
# define hprameters for both models

In [15]:
num_epochs = 10
lr = 0.001
batch_size = 150
tf_optim = keras.optimizers.Adam(learning_rate = lr)
tf_loss  = keras.losses.categorical_crossentropy
pt_optim = torch_optims.Adam(pt_model.parameters() , lr = lr)
pt_loss  = nn.CrossEntropyLoss()

In [16]:
# Prepare data for TF

In [17]:
(x_train , y_train) , (x_test , y_test) = data_tf()

x train shape =  (50000, 32, 32, 3)
x test  shape =  (10000, 32, 32, 3)
y train shape =  (50000, 10)
y test  shape =  (10000, 10)


In [18]:
#prepare data for PT

In [19]:
train_loader , test_loader = data_pt(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
# Train TF with CIFAR

In [21]:
tf_model.compile(loss = tf_loss , metrics = ["accuracy"] , optimizer = tf_optim) 
tf_model.fit(x = x_train , y = y_train , batch_size = batch_size , epochs = num_epochs , validation_data = (x_test , y_test))

Epoch 1/10
334/334 [==============================] - 4s 6ms/step - loss: 1.9580 - accuracy: 0.2831 - val_loss: 1.5945 - val_accuracy: 0.4229
Epoch 2/10
334/334 [==============================] - 1s 4ms/step - loss: 1.5322 - accuracy: 0.4467 - val_loss: 1.4116 - val_accuracy: 0.4893
Epoch 3/10
334/334 [==============================] - 1s 4ms/step - loss: 1.4204 - accuracy: 0.4876 - val_loss: 1.3998 - val_accuracy: 0.4924
Epoch 4/10
334/334 [==============================] - 1s 4ms/step - loss: 1.3389 - accuracy: 0.5220 - val_loss: 1.3218 - val_accuracy: 0.5254
Epoch 5/10
334/334 [==============================] - 1s 4ms/step - loss: 1.2810 - accuracy: 0.5413 - val_loss: 1.2992 - val_accuracy: 0.5295
Epoch 6/10
334/334 [==============================] - 1s 4ms/step - loss: 1.2340 - accuracy: 0.5610 - val_loss: 1.2472 - val_accuracy: 0.5523
Epoch 7/10
334/334 [==============================] - 1s 4ms/step - loss: 1.1971 - accuracy: 0.5751 - val_loss: 1.2582 - val_accuracy: 0.5525
Epoch 

In [22]:
tf_model.save("tf_model.h5")

In [23]:
#Train PT with CIFAR

In [30]:
def CalcAccuracy(generator , model):
  num_correct = 0  
  num_samples = 0
  with torch.no_grad():
    for x , y in generator:
      x = x.to(device = "cuda")
      y = y.to(device = "cuda")
      scores = model(x)
      _ , prediction = scores.max(1)
      num_correct += (prediction == y).sum()
      num_samples += prediction.size(0)
    acc = (num_correct / num_samples) * 100
  return acc

for epoch in range(num_epochs):
	for data , target in train_loader:
		data    = data.to(device = device)
		target  = target.to(device = device)
		scores  = pt_model(data)
		loss    = pt_loss(scores , target)
		pt_optim.zero_grad()
		loss.backward()
		pt_optim.step()
	train_acc   = CalcAccuracy(train_loader , pt_model)
	test_acc   = CalcAccuracy(test_loader  , pt_model)
	print("epoch {} train_acc = {:.3f} , test_acc = {:.3f}".format(epoch , train_acc , test_acc))

epoch 0 train_acc = 51.574 , test_acc = 50.800
epoch 1 train_acc = 53.432 , test_acc = 52.490
epoch 3 train_acc = 55.656 , test_acc = 53.950
epoch 5 train_acc = 56.166 , test_acc = 54.140
epoch 6 train_acc = 57.598 , test_acc = 55.780
epoch 7 train_acc = 58.464 , test_acc = 56.580
epoch 8 train_acc = 58.114 , test_acc = 56.300
epoch 9 train_acc = 58.878 , test_acc = 56.320
